In [59]:
import os

from langchain_community.utilities import SQLDatabase
from langchain.chains.openai_tools import create_extraction_chain_pydantic
from pydantic import BaseModel, Field
# from langchain_ollama.llms import OllamaLLM
from langchain_experimental.llms.ollama_functions import OllamaFunctions


In [119]:
OLLAMA_BASE_URL = "http://host.docker.internal:11434"
OLLAMA_MODEL_ID = "llama3.1"

# Connect to DB

In [120]:
db = SQLDatabase.from_uri("sqlite:///Chinook.db")
print(db.dialect)
print(db.get_usable_table_names())
db.run("SELECT * FROM Artist LIMIT 10;")

sqlite
['Album', 'Artist', 'Customer', 'Employee', 'Genre', 'Invoice', 'InvoiceLine', 'MediaType', 'Playlist', 'PlaylistTrack', 'Track']


"[(1, 'AC/DC'), (2, 'Accept'), (3, 'Aerosmith'), (4, 'Alanis Morissette'), (5, 'Alice In Chains'), (6, 'Antônio Carlos Jobim'), (7, 'Apocalyptica'), (8, 'Audioslave'), (9, 'BackBeat'), (10, 'Billy Cobham')]"

# Setup Langchain Ollama LLM

In [121]:
llm = OllamaFunctions(model=OLLAMA_MODEL_ID, base_url=OLLAMA_BASE_URL, verbose=True)

In [122]:
llm.invoke("Hi")

AIMessage(content='Hello! How are you today?', additional_kwargs={}, response_metadata={}, id='run-1c275633-f65e-44d1-875c-9acaf844c8ee-0')

# Setup Prompt to Select Potentially Relevant Tables Based on Query

In [129]:
class Tables(BaseModel):
    """Table in SQL database."""

    table_names: list[str] = Field(description="List of names of relevant tables in SQL database.")

In [130]:
table_names

'Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, Track'

In [132]:
table_names = ", ".join(db.get_usable_table_names())
prompt_template = f"""Below are the available table names:

{table_names}

Return the names of ALL the SQL tables that MIGHT be relevant to the user question.
Remember to include ALL POTENTIALLY RELEVANT tables, even if you're not sure that they're needed.

Query:
```sql
{{query}}
```"""
structured_model = llm.with_structured_output(Tables, include_raw=True)

prompt = prompt_template.format(query="What are all the genres of Alanis Morisette songs")
output = structured_model.invoke(prompt)

In [133]:
output["parsed"].model_dump()

{'table_names': ['Album', 'Artist', 'Genre', 'Track']}